In [38]:
import requests
import pandas as pd
import numpy as np
from keys import *

city = "Tokyo"
country = "JPN"
response = requests.get(f'http://api.openweathermap.org/data/2.5/forecast/?q={city},{country}&appid={OWM_key}&units=metric&lang=en')
print(response.json())

##En este codigo nos da como resultados la temperatura, la temperatura min, max, humedad, precion del aire, etc

{'cod': '200', 'message': 0, 'cnt': 40, 'list': [{'dt': 1762074000, 'main': {'temp': 18.49, 'feels_like': 17.94, 'temp_min': 18.49, 'temp_max': 19.66, 'pressure': 1010, 'sea_level': 1010, 'grnd_level': 1008, 'humidity': 59, 'temp_kf': -1.17}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 75}, 'wind': {'speed': 1.54, 'deg': 308, 'gust': 1.76}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-11-02 09:00:00'}, {'dt': 1762084800, 'main': {'temp': 18.54, 'feels_like': 17.91, 'temp_min': 18.54, 'temp_max': 18.65, 'pressure': 1010, 'sea_level': 1010, 'grnd_level': 1009, 'humidity': 56, 'temp_kf': -0.11}, 'weather': [{'id': 803, 'main': 'Clouds', 'description': 'broken clouds', 'icon': '04n'}], 'clouds': {'all': 78}, 'wind': {'speed': 1.72, 'deg': 313, 'gust': 2.31}, 'visibility': 10000, 'pop': 0, 'sys': {'pod': 'n'}, 'dt_txt': '2025-11-02 12:00:00'}, {'dt': 1762095600, 'main': {'temp': 17.67, 'feels_like': 17.0

In [39]:
data = response.json()

# Extract forecast list
forecast_list = data.get('list', [])

# Prepare lists for DataFrame columns
times = []
temperatures = []
humidities = []
weather_statuses = []
wind_speeds = []
rain_volumes = []
snow_volumes = []

for entry in forecast_list:
    times.append(entry.get('dt_txt', np.nan))
    temperatures.append(entry.get('main', {}).get('temp', np.nan))
    humidities.append(entry.get('main', {}).get('humidity', np.nan))
    weather_statuses.append(entry.get('weather', [{}])[0].get('main', np.nan))
    wind_speeds.append(entry.get('wind', {}).get('speed', np.nan))
    rain_volumes.append(entry.get('rain', {}).get('3h', np.nan))
    snow_volumes.append(entry.get('snow', {}).get('3h', np.nan))

# Create DataFrame
df = pd.DataFrame({
    'weather_datetime': times,
    'temperature': temperatures,
    'humidity': humidities,
    'weather_status': weather_statuses,
    'wind': wind_speeds,
    'rain_qty': rain_volumes,
    'snow': snow_volumes,
    'municipality_iso_country': f"Tokyo,JPN"
})

print(df.head())

## Este codigo actua como el anterior pero esta vez nos los da como en forma de lista 


      weather_datetime  temperature  humidity weather_status  wind  rain_qty  \
0  2025-11-02 09:00:00        18.49        59         Clouds  1.54       NaN   
1  2025-11-02 12:00:00        18.54        56         Clouds  1.72       NaN   
2  2025-11-02 15:00:00        17.67        60         Clouds  1.44       NaN   
3  2025-11-02 18:00:00        16.28        67         Clouds  1.93       NaN   
4  2025-11-02 21:00:00        15.63        66          Clear  2.21       NaN   

   snow municipality_iso_country  
0   NaN                Tokyo,JPN  
1   NaN                Tokyo,JPN  
2   NaN                Tokyo,JPN  
3   NaN                Tokyo,JPN  
4   NaN                Tokyo,JPN  


In [40]:
import sqlalchemy
import pymysql
# connection details for the local mysql database
schema = "gans"
host = "127.0.0.1"
user = "root"
password = "Ua-xxx99"
port = 3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [41]:
# send the weather data to the database
df.to_sql('weather_data', if_exists = 'append', con = con, index=False)

40